<a href="https://colab.research.google.com/github/Satwikram/Deep-Learning-Notebooks/blob/master/Transfer%20Learning%20and%20Fine%20Tuning/Monkey_Breed_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
@Author: Satwik Ram K
"""
# Importing Dependecies
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:
from google.colab import files

files.upload()

! mkdir ~/.kaggle


! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [4]:
!kaggle datasets download -d slothkong/10-monkey-species #Copy API command and execute

 95% 521M/547M [00:07<00:00, 90.4MB/s]
100% 547M/547M [00:07<00:00, 75.2MB/s]


In [ ]:
!unzip /content/10-monkey-species.zip

Using Transfer Learning to classify monkey breed

In [6]:
from tensorflow.keras.applications import MobileNet

In [8]:
img_size = (224, 224, 3)


Load the mobilenet net model without top layer or FC 

In [9]:
Mobilenet = MobileNet(weights = 'imagenet', include_top = False, input_shape = img_size )

17227776/17225924 [==============================] - 0s 0us/step


In [46]:
# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layers in Mobilenet.layers:
  layers.trainable = False


In [47]:
# Lets Print out our layers
for (i, layer) in enumerate(Mobilenet.layers):
  print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False
45 BatchNormalization False
46 ReLU False
47 Conv2D False
48 Batc

# Let's make a function that returns our FC Head

In [48]:
def addTopModel(bottom_model, num_classes):
  """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

  top_model = bottom_model.output
  top_model = GlobalAveragePooling2D()(top_model)
  top_model = Dense(1024, activation = 'relu')(top_model)
  top_model = Dense(1024, activation = 'relu')(top_model)
  top_model = Dense(512,activation='relu')(top_model)
  top_model = Dense(num_classes,activation='softmax')(top_model)

  return top_model



Let's add our FC Head back onto MobileNet

In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model

In [50]:
# Set number of classes
num_classes = 10

# FC Head
FC_Head = addTopModel(Mobilenet, num_classes)

In [51]:
model = Model(inputs = Mobilenet.input, outputs = FC_Head)


In [52]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

Loading our Monkey Breed Dataset

In [53]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [54]:
train_data_dir = '/content/training/training'
val_data_dir = '/content/validation/validation'

Data Augmentation

In [55]:
train_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest' 
)

In [56]:
validation_datagen = ImageDataGenerator(rescale=1./255)


In [57]:
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32

In [70]:
img_size[:2]

(224, 224)

In [71]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = img_size[:2],
    batch_size = batch_size,
    class_mode='categorical'
)

Found 1098 images belonging to 10 classes.


In [72]:
validation_generator = validation_datagen.flow_from_directory(
        val_data_dir,
        target_size= img_size[:2],
        batch_size=batch_size,
        class_mode='categorical')

Found 272 images belonging to 10 classes.


Training Model

In [73]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

Creating Checkpoints after every epochs

In [74]:
checkpoint = ModelCheckpoint('monkey.h5', monitor = 'vall_loss', mode = 'min', save_best_only = True, verbose = 1)

Creating EarlyStopping

In [75]:
earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)


In [76]:
# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]


In [77]:
# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

In [78]:
# Enter the number of training and validation samples here
nb_train_samples = 1098
nb_validation_samples = 272

batch_size = 16
epochs = 5

In [79]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/5
68/68 [==============================] - 92s 1s/step - loss: 2.0456 - accuracy: 0.4593 - val_loss: 0.4180 - val_accuracy: 0.8419
Epoch 2/5
68/68 [==============================] - 92s 1s/step - loss: 0.6399 - accuracy: 0.7957 - val_loss: 1.8412 - val_accuracy: 0.6801
Epoch 3/5
68/68 [==============================] - 91s 1s/step - loss: 0.5839 - accuracy: 0.8401 - val_loss: 0.5779 - val_accuracy: 0.8676
Epoch 4/5
68/68 [==============================] - 91s 1s/step - loss: 0.5091 - accuracy: 0.8586 - val_loss: 0.9448 - val_accuracy: 0.7426
Epoch 00004: early stopping
